## 24.2 Single-source shortest paths in directed acyclic graphs

### 24.2-1

> Run DAG-SHORTEST-PATHS on the directed graph of Figure 24.5, using vertex $r$ as the source.

|\|$r$|$s$|$t$|$x$|$y$|$z$|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|$d$|0|5|3|10|7|5|
|$\pi$|NIL|$r$|$r$|$t$|$t$|$t$|

### 24.2-2

> Suppose we change line 3 of DAG-SHORTEST-PATHS to read 
> 
> 3  __for__ the first $|V| - 1$ vertices, taken in topologically sorted order
> 
> Show that the procedure would remain correct.

The out-degree of the last vertex is 0.

### 24.2-3

> The PERT chart formulation given above is somewhat unnatural. In a more natural structure, vertices would represent jobs and edges would represent sequencing constraints; that is, edge $(u, v)$ would indicate that job $u$ must be performed before job $v$. We would then assign weights to vertices, not edges. Modify the DAG-SHORTEST- PATHS procedure so that it finds a longest path in a directed acyclic graph with weighted vertices in linear time.

$s.d = s.w$, $w(u, v) = v.w$.

### 24.2-4

> Give an efficient algorithm to count the total number of paths in a directed acyclic graph. Analyze your algorithm.

$s.num = 1$, $v.num = \sum_{(u, v) \in E} u.num$.

Time: $\Theta(V + E)$.

# Hw

### BellmanFord Algorithm 

기본가정: no negative weight cycle 

running time: 
T = O(|V||E|)

In [5]:
# node로 구현 
class node: 
    
    def __init__(self, name):
        self.name = name
        self.d = 0 
        self.f = 0
        self.pi = 0  
        self.key = 0
        self.color = 'unknown'
        self.edge = {}
        
class Graph: 
    
    time = 0
    
    def __init__(self, graph_dict=None, w = None):
        if graph_dict == None:
            graph_dict = {}
        self.graph_dict = graph_dict
        
        # node object로 list 만듦
        self.nodes = []
        for n in list(self.graph_dict.keys()):
            self.nodes.append(node(n))
        
        # node object로 dictionary 만듦
        self.nm = {}
        for k in range(len(list(self.graph_dict.keys()))):
            self.nm[self.nodes[k].name] = self.nodes[k]
        
        # edge 를 w 에 의해 부여 
        for j in range(len(list(self.graph_dict.keys()))):  # j vertex
            for i in self.graph_dict[self.nodes[j].name]:      # i edge 
                self.nodes[j].edge[i] = w[self.nodes[j].name][i]             # allocate weight 
                
    
    # sort the edge of G.E into nondecreasing order by weight w 
    def sort_edge(self):
        import operator
        temp = []
    
        for v in list(self.graph_dict.keys()):
            for i in list(self.nm[v].edge):
                temp.append((v,i,self.nm[v].edge[i]))

        t = sorted(temp, key = operator.itemgetter(2), reverse = False)
        #print(t)

        srt = []
        for m in range(len(t)): 
            srt.append([t[m][0],t[m][1]])
        #print(srt,"sort 이후 edge")
        return srt            
                
    
    
    """ single src shortest path operation """
    def initialize_single_source(self,s):
        for v in list(self.graph_dict.keys()):
            self.nm[v].d = float('inf')
            self.nm[v].pi = None
        self.nm[s].d = 0
        # T = O(V)

    def relax(self,u,v):
        if self.nm[v].d > self.nm[u].d + self.nm[u].edge[v]:
            self.nm[v].d = self.nm[u].d + self.nm[u].edge[v]
            self.nm[v].pi = self.nm[u].name
        # T = O(1)
        
    def showinfo(self):
        print(self.graph_dict)
        print(" node dictionary : ")
        print(self.nm)
        
        for v in list(G.graph_dict.keys()):
            print(">> [ ",v, " node Info] : ")
            print(v, ", edge: ", G.nm[v].edge)
            print(v, ", pi: ", G.nm[v].pi)
            print(v, ", d: ", G.nm[v].d)

In [8]:
def BellmanFord(G,s):
    srt = G.sort_edge()
    
    G.initialize_single_source('s')
    
    """ Finding the single src shortest parth from s to each vertex"""
    # iteration |V| - 1 
    for i in range(len(G.graph_dict.keys())):           # O(V) * 
        # each edge (u,v) ∈ G.E
        for [u,v] in srt:                               # O(E)
               G.relax(u,v)           
        
    # test whether ∃ negative weight cycle 
    for [u,v] in srt:
        if G.nm[v].d > G.nm[u].d + G.nm[u].edge[v]:
            return False
    
    # T = O(|V||E|)
    return True
            
     

In [14]:
vertices = ['s','t','x','y','z']
edges = [{'t','y'},{'x','y','z'},{'t'},{'x','z'},{'s','x'}]
weights = [{'t': 6, 'y': 7 }, {'x': 5  ,'y': 8 ,'z': -4 }, {'t': -2 }, {'x': -3 ,'z': 9 }, {'s': 2 ,'x': 7 }]
g = {}
w = {}
k = 0
for v in (vertices):
    g[v] = edges[k]
    w[v] = weights[k]  
    k = k + 1

G = Graph(g,w)
print("※ BellmanFord Algorithm result : ", BellmanFord(G,'s'))
G.showinfo()

※ BellmanFord Algorithm result :  True
{'z': {'s', 'x'}, 's': {'t', 'y'}, 't': {'y', 'z', 'x'}, 'y': {'z', 'x'}, 'x': {'t'}}
 node dictionary : 
{'s': <__main__.node object at 0x000000A10B8A1080>, 'x': <__main__.node object at 0x000000A10B8A1208>, 't': <__main__.node object at 0x000000A10B8A1F28>, 'z': <__main__.node object at 0x000000A10B8A18D0>, 'y': <__main__.node object at 0x000000A10B8A15F8>}
>> [  z  node Info] : 
z , edge:  {'s': 2, 'x': 7}
z , pi:  t
z , d:  -2
>> [  s  node Info] : 
s , edge:  {'t': 6, 'y': 7}
s , pi:  None
s , d:  0
>> [  t  node Info] : 
t , edge:  {'x': 5, 'y': 8, 'z': -4}
t , pi:  x
t , d:  2
>> [  y  node Info] : 
y , edge:  {'z': 9, 'x': -3}
y , pi:  s
y , d:  7
>> [  x  node Info] : 
x , edge:  {'t': -2}
x , pi:  y
x , d:  4
